<a href="https://colab.research.google.com/github/Jolayemi-momoh/miniature-bassoon/blob/main/Financial_health_Data_cleaning_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
train_df=pd.read_csv('/content/drive/MyDrive/dataorg-financial-health-prediction-challenge20251204-19827-m2tn1n/Train.csv')
test_df=pd.read_csv('/content/drive/MyDrive/dataorg-financial-health-prediction-challenge20251204-19827-m2tn1n/Test.csv')

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
if 'ID' in train_df.columns:
    train_df.drop('ID', axis=1, inplace=True)

In [ ]:
if 'business_age_years' in train_df.columns and 'business_age_months' in train_df.columns:
  missing_age_mask = train_df['business_age_years'].isna() & train_df['business_age_months'].isna()
  train_df['business_age'] = (train_df['business_age_years'].fillna(0) * 12) + train_df['business_age_months'].fillna(0)
  train_df.loc[missing_age_mask, 'business_age'] = np.nan
  train_df.drop(columns=['business_age_years', 'business_age_months'], inplace=True)

In [ ]:
if 'business_age_years' in test_df.columns and 'business_age_months' in test_df.columns:
  missing_age_mask = test_df['business_age_years'].isna() & test_df['business_age_months'].isna()
  test_df['business_age'] = (test_df['business_age_years'].fillna(0) * 12) + test_df['business_age_months'].fillna(0)
  test_df.loc[missing_age_mask, 'business_age'] = np.nan
  test_df.drop(columns=['business_age_years', 'business_age_months'], inplace=True)

In [ ]:
num_cols=train_df.select_dtypes(include=['number']).columns
cat_cols=train_df.select_dtypes(include=['object']).columns

In [ ]:
train_df['Target'].dropna().unique()

In [ ]:
for col in num_cols:
  train_df[col].fillna(train_df[col].median(),inplace=True)
  test_df[col].fillna(test_df[col].median(),inplace=True)

In [ ]:
for col in cat_cols:
  train_df[col].fillna('Unknown',inplace=True)
  if col in test_df.columns:
    test_df[col].fillna('Unknown',inplace=True)

In [ ]:
train_df.isna().sum().sum()


In [ ]:
test_df.isna().sum().sum()

In [ ]:
train_df.isna().sum().sum()

In [ ]:
train_df.replace({'’': "'", '\?': "'"}, regex=True)

In [ ]:
dont_know_variants=[
    "Don't Know",
    "Do not know/N/A",
    "Don't know /doesn't apply",
    "Don't know(Do not show)",
    "Don't know or N/A"
]
train_df=train_df.replace(dont_know_variants, "Don't know")
test_df=test_df.replace(dont_know_variants, "Don't know")


In [ ]:
if "current_problem_cash_flow" in train_df.columns and "current_problem_cash_flow" in test_df.columns:
  train_df['current_problem_cash_flow']=train_df['current_problem_cash_flow'].replace({'0':'No'})
  test_df['current_problem_cash_flow']=test_df['current_problem_cash_flow'].replace({'0':'No'})

In [ ]:
if 'keeps_financial_records' in train_df.columns and 'keeps_financial' in test_df.columns:
    train_df['keeps_financial_records']=train_df['keeps_financial_records'].replace({'yes': 'yes, sometimes'})
    test_df['keeps_financial']=test_df['keeps_financial'].replace({'yes': 'yes, sometimes'})

In [ ]:
# Isolate ages 80 and above, count them, and sort by the age itself
older_ages = test_df.loc[test_df['owner_age'] >= 80, 'owner_age']
print(older_ages.value_counts().sort_index())


In [ ]:

# Convert the survey dummy codes to actual missing values
train_df['owner_age'] = train_df['owner_age'].replace([99.0, 103.0], np.nan)

In [ ]:
train_df['owner_age'] = train_df['owner_age'].fillna(train_df['owner_age'].median())

In [ ]:
train_df.describe()

In [ ]:
# Look at the jump between the 99% mark and the absolute maximum

for col in num_cols:
    print(f"--- {col.upper()} ---")
    print(f"Median (50%):         {train_df[col].median():,.2f}")
    print(f"95th Percentile:      {train_df[col].quantile(0.95):,.2f}")
    print(f"99th Percentile:      {train_df[col].quantile(0.99):,.2f}")
    print(f"Absolute MAX:         {train_df[col].max():,.2f}")
    print("\n")


In [ ]:
exchange_rates_to_usd={
    "eswani":19.0,
    'lesotho':19.0,
    "malawi":1700,
    'zimbabwe':1
}

In [ ]:
train_rates_series = train_df['country'].map(exchange_rates_to_usd)
test_rates_series = test_df['country'].map(exchange_rates_to_usd)

In [ ]:
financial_cols = ['personal_income', 'business_expenses', 'business_turnover']

In [ ]:
for col in financial_cols:
  if col in train_df.columns and col in test_df.columns:
    usd_col_name=f'{col}_usd'
    train_df[usd_col_name]=train_df[col]/train_rates_series
    test_df[usd_col_name]=test_df[col]/test_rates_series
    cap_value=train_df[usd_col_name].quantile(0.99)
    train_df[usd_col_name]=train_df[usd_col_name].clip(upper=cap_value)
    test_df[usd_col_name]=test_df[usd_col_name].clip(upper=cap_value)
    train_df=train_df.drop(columns=[col],inplace=True)
    test_df=test_df.drop(columns=[col], inplace=True)

In [ ]:
train_df.describe()

In [ ]:
train_df.to_csv('cleaned_train.csv',index=False)
test_df.to_csv('cleaned_test.csv',index=False)